In [7]:
import glob
import numpy as np
import pandas as pd
from datetime import  date
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import os
from talib import RSI, BBANDS

import talib


In [8]:
data_path =  os.path.join(os.getcwd(),"nse_data\\data\\")
data_index = os.path.join(os.getcwd(),"nse_data\\index_data\\")
dump_path =  os.path.join(os.getcwd(),"nse_data\\dump\\")
meta_data =  os.path.join(os.getcwd(),"nse_data\\meta_data\\")
reports_path = os.path.join(os.getcwd(),"nse_data\\reports\\")
nse_indices = os.path.join(os.getcwd(),"nse_data\\nse_indices\\")
# print(nse_indices)

In [9]:
nifty500_df = pd.read_csv(os.path.join(nse_indices, "ind_nifty500list.csv"))
stocks_list = nifty500_df.Symbol.tolist()
len(stocks_list)

501

In [10]:
stocks_list = list(dict.fromkeys(stocks_list))

add_stocks = ['VEDL', 'YESBANK', 'SBICARD']
skip_stocks = ['BBTC','IRB','M&MFIN','PVR','HATHWAY']

stocks_list = stocks_list + add_stocks
stocks_list = sorted(set(stocks_list)-(set(skip_stocks)))
len(stocks_list)

# stocks_list = random.sample(stocks_list, len(stocks_list))

498

In [5]:
start_date = '2020-08-07'
end_date = date.today()
month = 8

In [6]:
# Function to get script data file path if present
def get_stock_file_path(script):
    for file in glob.glob(data_path + script + '_*.csv'):
        file = file.replace("\\","/")
        return(file)
    
def get_stock_filename(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)
    
# Function to check if the script data file already present or not: True or False
def stock_data_present(script):
    file = get_stock_file_path(script)
    if file:
        return True
    return False

# Function to get stock dataframe from script name
def stock_df_from_csv(script):
    file = get_stock_file_path(script)
    stock_df = pd.read_csv(file)
    return stock_df

# Function to get stock dataframe from script name with Date as Index
def get_df_date_index(stock):
    df = stock_df_from_csv(stock)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', drop=True, inplace = True)
    df.index.name = None
    return df



In [7]:
# GET ALL DATES 

def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    return next_month - datetime.timedelta(days=next_month.day)

def first_day_of_month(any_day):
    return any_day.replace(day=1)


df = get_df_date_index("TCS")
# month_start_date = date.today()
# print(month_start_date.month)


last_updated_date = df.index.max()

last_day_month = last_day_of_month(last_updated_date)
first_day_month = first_day_of_month(last_updated_date)

print("Last Updated Date:", last_updated_date)
print("First Month Date:",first_day_month)
print("Last Month Date:",last_day_month)

Last Updated Date: 2020-09-04 00:00:00
First Month Date: 2020-09-01 00:00:00
Last Month Date: 2020-09-30 00:00:00


In [8]:
# Find top Gainers and Top Losers
no_of_values = 10
day_change = []


for stock in stocks_list:
    df = stock_df_from_csv(stock)
    df = df.tail(1)
    change = (df.Close - df['Prev Close'])*100 / df['Prev Close']
    change = float(round(change,2))
    day_change.append({
                        'Stock':stock,
                        "Change_Percent":change
    })

df = pd.DataFrame(day_change)
df = df.sort_values("Change_Percent", ascending = False)
df = pd.concat([df.head(no_of_values) , df.tail(no_of_values)])

datetime_obj = last_updated_date.to_pydatetime().date() 
gainer_loser_file = reports_path + 'Gainer_Loser_Report_' + datetime_obj.strftime('%d-%m-%Y') + '.csv'
df.to_csv(gainer_loser_file, index = True)
print(gainer_loser_file, ' Saved to Disk')

C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\reports\Gainer_Loser_Report_04-09-2020.csv  Saved to Disk


In [23]:
# df

In [24]:
# kkk = df.copy()
# print(type(kkk))
# def highlight_cols(y):
#     #copy df to new - original data are not changed
#     print(type(y))
#     x = y.copy()
#     print(type(x))
#     x[x['Change_Percent']>0] = 'background-color: green'
# #     x[x["Change Percent"]<=0] = 'background-color: red'
# #     x[x["Change Percent"]==0] = 'background-color: yellow'
#     return x 

# k = kkk.style.apply(highlight_cols, axis=1)
# print(k)
# datetime_obj = last_updated_date.to_pydatetime().date() 
# gainer_loser_file = reports_path + 'Gainer_Loser_Report_' + datetime_obj.strftime('%d-%m-%Y') + '.xlsx'
# k.to_excel(gainer_loser_file, index = True)
# print(gainer_loser_file, ' Saved to Disk')

In [25]:
# for month in range(1, 13):
#     print(last_day_of_month(datetime.date(2020, month, 1)))